# MODEL TRAINING - testing the full approach - DREAMS
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## experiment id
we'll use it to create a folder at the end of the script

In [2]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate

In [3]:
samplerate=200

## load dreams

In [4]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

764

In [6]:
minDuration=0.3
maxDuration=2.9
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

764

In [7]:
annotations.head()

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd
0,0,282.24,0.72,C3-A1,0001,0001,spindle,200,282.96,56448,56592
1,1,311.72,1.54,C3-A1,0001,0001,spindle,200,313.26,62344,62652
2,2,340.28,0.72,C3-A1,0001,0001,spindle,200,341.00,68056,68200
3,3,366.83,0.65,C3-A1,0001,0001,spindle,200,367.48,73366,73496
4,4,373.74,0.50,C3-A1,0001,0001,spindle,200,374.24,74748,74848


In [8]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS


## define a fixed feature selection to use

In [9]:
featureSelection=loadPickle("featureSelection26.pkl")
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## load data split
LOOCV using 5 subjects, 1 for validation

In [10]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_LOOCV_5S.pkl")
dataSplits

,train,val,test
0,"[0003, 0005, 0006]",[0002],0001
1,"[0001, 0003, 0005]",[0006],0002
2,"[0001, 0002, 0006]",[0005],0003
3,"[0001, 0002, 0006]",[0003],0005
4,"[0001, 0003, 0005]",[0002],0006


## define annotation criteria
we want to test the models in three diferent ways

In [11]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001','0002','0003'],
    'criteriumName':['E1','E2','union'],
    'labelerIdList':[['0001'],['0002'],['0001','0002']]
})

In [12]:
annotationCriteria

,criteriumId,criteriumName,labelerIdList
0,0001,E1,[0001]
1,0002,E2,[0002]
2,0003,union,"[0001, 0002]"


In [13]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [14]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test
0,0001,E1,[0001],"[0003, 0005, 0006]",[0002],0001
1,0001,E1,[0001],"[0001, 0003, 0005]",[0006],0002
2,0001,E1,[0001],"[0001, 0002, 0006]",[0005],0003
3,0001,E1,[0001],"[0001, 0002, 0006]",[0003],0005
4,0001,E1,[0001],"[0001, 0003, 0005]",[0002],0006
5,0002,E2,[0002],"[0003, 0005, 0006]",[0002],0001
6,0002,E2,[0002],"[0001, 0003, 0005]",[0006],0002
7,0002,E2,[0002],"[0001, 0002, 0006]",[0005],0003
8,0002,E2,[0002],"[0001, 0002, 0006]",[0003],0005
9,0002,E2,[0002],"[0001, 0003, 0005]",[0002],0006


## model fitting

In [15]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.4
subsample=0.6

num_boost_round=60  #number of boosting rounds per model

models=[]
modelIds=[]
valF1s=[]
spindleTimeRates=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,datapath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)
    
    #Train the models
    params={
        'n_jobs':n_jobs,
        'learning_rate':learning_rate,
        'subsample':subsample,
        'objective':'binary:logistic'
    }
    trainDMatrix=xgb.DMatrix(data=trainFeatures,label=trainLabels)
    xgb_model = xgb.train(params,trainDMatrix,num_boost_round=num_boost_round)
    spindleTimeRates.append(sum(trainLabels)/len(trainLabels))
    models.append(xgb_model)
    modelIds.append(str(uuid.uuid4()))
    

**********************
1 of 15
**********************
2 of 15
**********************
3 of 15
**********************
4 of 15
**********************
5 of 15
**********************
6 of 15
**********************
7 of 15
**********************
8 of 15
**********************
9 of 15
**********************
10 of 15
**********************
11 of 15
**********************
12 of 15
**********************
13 of 15
**********************
14 of 15
**********************
15 of 15


In [16]:
experimentModels["modelId"]=modelIds
experimentModels["spindleTimeRate"]=spindleTimeRates

In [17]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0006]",[0002],0001,83b7a320-06e0-4b5f-a99f-7863b21ac84b,0.018348
1,0001,E1,[0001],"[0001, 0003, 0005]",[0006],0002,de0568df-657a-4cef-b14e-7d0cae36ca8e,0.016172
2,0001,E1,[0001],"[0001, 0002, 0006]",[0005],0003,3bcdc5c8-b4d4-4f58-a892-fcc66da68ef3,0.026374
3,0001,E1,[0001],"[0001, 0002, 0006]",[0003],0005,3c4bdd12-6869-41a4-a9b0-70c68730415d,0.026374
4,0001,E1,[0001],"[0001, 0003, 0005]",[0002],0006,a2fe24ce-b6bb-43f9-a58e-b25f9e3630bc,0.016172
5,0002,E2,[0002],"[0003, 0005, 0006]",[0002],0001,f96c5ab5-f40f-4ed2-a2b7-286126d165bc,0.040185
6,0002,E2,[0002],"[0001, 0003, 0005]",[0006],0002,48a66f14-7df2-41f8-9fcd-8145e1deb173,0.045183
7,0002,E2,[0002],"[0001, 0002, 0006]",[0005],0003,6e78e3c9-8c7b-417e-8863-47b5a52a32dc,0.046850
8,0002,E2,[0002],"[0001, 0002, 0006]",[0003],0005,9d3f65cd-baee-4a36-a05a-0b8dbc875830,0.046850
9,0002,E2,[0002],"[0001, 0003, 0005]",[0002],0006,76a1415b-361b-4e47-9fb6-845d93d0e4d1,0.045183


## save results

In [18]:
#create parent folder
experimentpath=datapath+"/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for id, model in zip(modelIds,models):
    model.save_model(experimentpath+"/"+id+".json")
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)
dumpPickle(experimentpath+"/featureSelection.pkl",featureSelection)

In [19]:
#you need to use this in the following parts of the experiment
print(experimentId)

68877408-cc71-4aee-8777-09b36e149a50
